In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import sklearn
from skimage.feature import greycomatrix, greycoprops
from sklearn.preprocessing import StandardScaler

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

print(np.shape(x_train))
print(np.shape(x_test))

(60000, 28, 28)
(10000, 28, 28)


In [ ]:
def img_glcm(x):
  img_glcm = []
  distances = [1, 3, 5, 10, 20]

  for img in x:
    temp = []
    for dis in distances:
      glcm = greycomatrix(img, distances=[dis], angles= [0], levels= 256, symmetric=True, normed=False)
      temp.extend([greycoprops(glcm, 'dissimilarity')[0, 0], greycoprops(glcm, 'contrast')[0, 0], greycoprops(glcm, 'correlation')[0, 0], greycoprops(glcm, 'homogeneity')[0, 0], greycoprops(glcm, 'energy')[0, 0]])
    
    img_glcm.append(temp)
  return img_glcm

In [ ]:
img_glcm_train = img_glcm(x_train)

In [ ]:
img_glcm_test = img_glcm(x_test)

In [ ]:
print(np.shape(img_glcm_train))
print(np.shape(img_glcm_test))

(60000, 25)
(10000, 25)


In [ ]:
print(img_glcm_train[1])

[22.886243386243386, 3279.6349206349205, 0.7734203525644802, 0.756054291666285, 0.7280048843730417, 59.07142857142857, 12232.85714285714, 0.20256010388417073, 0.6345753558546408, 0.6237534507695643, 77.22981366459626, 16331.468944099379, -0.0018647559758562086, 0.5484597348091829, 0.5429111948444624, 77.04365079365078, 16570.72619047619, -0.059870906112974166, 0.5500288172459197, 0.5523094650193061, 39.558035714285715, 8593.004464285714, -0.10017419409327868, 0.7725882578308524, 0.7738617484666089]


PCA-data


In [ ]:
import numpy as np
e = 1e-9
imag_mean = np.mean(img_glcm_train,axis = 0, keepdims=True)
imag_std = np.std(img_glcm_train,axis = 0, keepdims=True)
img_glcm_train -= imag_mean;   img_glcm_train /= (imag_std + e)   
img_glcm_test  -= imag_mean;   img_glcm_test /= (imag_std + e)

In [ ]:
from sklearn import decomposition
n_components = 16
pca = decomposition.PCA(n_components=n_components)

In [ ]:
# fit data
pca.fit(img_glcm_train)
Z_train = pca.transform(img_glcm_train)
Z_test  = pca.transform(img_glcm_test)

In [ ]:
print(Z_train[1])
print(Z_train.shape)
Z_test.shape

[-4.1852043  -2.30147485 -1.30203685  0.19469035  0.45210201 -0.23673819
  0.52361553 -0.12210081  0.43750313  0.05175279  0.10049139 -0.02378572
 -0.03784972 -0.06901242 -0.19596843 -0.04370894]
(60000, 16)


(10000, 16)

n-bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_model.fit(Z_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = nb_model.predict(Z_test)
print(accuracy_score(y_test,y_pred))

0.5842


neural network

In [ ]:
import tensorflow as tf
from tensorflow.python.eager.context import graph_mode
# xây dụng model 
model = tf.keras.models.Sequential()
#chuyển dữ liệu về dạng phẳng 28x28 = 1x748
# model.add(tf.keras.layers.Flatten())
#tạo ra 3 lớp của model với 128 điểm  
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
#tạo lớp cuối vói 10 điểm 
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

In [ ]:
# compile model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(Z_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5210 - accuracy: 0.8053
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5103 - accuracy: 0.8096
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5029 - accuracy: 0.8119
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4984 - accuracy: 0.8138
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4896 - accuracy: 0.8164


Epoch 1/5


In [ ]:
#loss function and accuracy function
val_loss, val_acc = model.evaluate(Z_test, y_test)
print(val_acc)

313/313 [==============================] - 1s 2ms/step - loss: 0.7977 - accuracy: 0.7475
0.7475000023841858


k-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# fit model
knn = KNeighborsClassifier(n_neighbors=100, p=2)
knn.fit(Z_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=100, p=2,
                     weights='uniform')

In [ ]:
y_pred = knn.predict(Z_test)
print(accuracy_score(y_test,y_pred))

0.6652


logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lgt = LogisticRegression()
lgt.fit(Z_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_predict = lgt.predict(Z_test)
accuracy = accuracy_score(y_predict,y_test)
print(accuracy)

0.6395
